In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import lightgbm as lg

In [2]:
# Reading the Dataset
dataset = pd.read_csv("Preprocessed_project_risk.csv")
dataset

,Team_Size,Project_Budget_USD,Estimated_Timeline_Months,Complexity_Score,Stakeholder_Count,Past_Similar_Projects,External_Dependencies_Count,Change_Request_Frequency,Team_Turnover_Rate,Vendor_Reliability_Score,...,Tech_Environment_Stability,Contract_Type,Resource_Contention_Level,Industry_Volatility,Client_Experience_Level,Change_Control_Maturity,Risk_Management_Maturity,Team_Colocation,Documentation_Quality,Risk_Level
0,32.0,1526276.550,32.0,9.70,16.0,3.0,3.0,1.05,0.16,0.84,...,0,Time & Materials,High,Extreme,First-time,Basic,Basic,Fully Colocated,Good,High
1,2.0,390790.150,9.0,2.72,9.0,0.0,2.0,2.61,0.42,0.79,...,0,Cost-Plus,Low,Stable,Occasional,Advanced,Formal,Fully Remote,Poor,Low
2,2.0,246674.760,6.0,2.04,7.0,1.0,0.0,0.83,0.55,0.89,...,0,Cost-Plus,High,Stable,Regular,0,0,Hybrid,Good,Medium
3,12.0,1427830.630,17.0,7.54,16.0,0.0,5.0,2.42,0.33,0.84,...,Mixed,Hybrid,High,Extreme,Strategic,Formal,Basic,Hybrid,Basic,High
4,24.0,1696746.640,24.0,6.68,17.0,0.0,2.0,0.16,0.36,0.86,...,0,Cost-Plus,High,Moderate,Occasional,Basic,0,Partially Colocated,Basic,High
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
3995,9.0,731548.970,14.0,5.40,9.0,4.0,3.0,1.09,0.25,0.94,...,Legacy/Unstable,Fixed-Price,Medium,Moderate,Occasional,0,0,Hybrid,Basic,High
3996,9.0,492981.400,9.0,2.69,8.0,5.0,3.0,2.11,0.06,0.71,...,0,Fixed-Price,Medium,Stable,Occasional,0,Basic,Hybrid,Basic,Medium
3997,12.0,1097040.650,24.0,8.82,10.0,2.0,4.0,0.52,0.23,0.95,...,Legacy/Unstable,Time & Materials,Low,Moderate,Regular,0,Basic,Fully Remote,Basic,High
3998,26.0,2650876.115,28.0,10.00,23.0,1.0,6.0,1.19,0.30,0.50,...,0,Fixed-Price,Medium,High,Strategic,Basic,Formal,Fully Remote,Good,High


In [3]:
# Separate target first
dependent = dataset["Risk_Level"]
independent = dataset.drop(columns=["Risk_Level"])

# Apply get_dummies only on independent
X_encoded = pd.get_dummies(independent, drop_first=True)
X_encoded=X_encoded.replace({True:1,False:0})
from sklearn.preprocessing import LabelEncoder

le = LabelEncoder()
Y_encoded = le.fit_transform(dependent)
print(X_encoded.shape)
print(Y_encoded.shape)

C:\Users\PRIYANKA MANI\AppData\Local\Temp\ipykernel_23988\4170489483.py:7: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  X_encoded=X_encoded.replace({True:1,False:0})


(4000, 4098)
(4000,)


In [4]:
#from sklearn.feature_selection import SelectKBest, f_classif

#selector = SelectKBest(score_func=f_classif, k=40)
#X_anova = selector.fit_transform(X_encoded, Y_encoded)

#selected_features = X_encoded.columns[selector.get_support()]
#X_selected = X_encoded[selected_features]
#X_selected

In [5]:
dependent

0         High
1          Low
2       Medium
3         High
4         High
         ...  
3995      High
3996    Medium
3997      High
3998      High
3999    Medium
Name: Risk_Level, Length: 4000, dtype: object

In [6]:
corr_matrix=X_encoded.corr()
corr_matrix

,Team_Size,Project_Budget_USD,Estimated_Timeline_Months,Complexity_Score,Stakeholder_Count,Past_Similar_Projects,External_Dependencies_Count,Change_Request_Frequency,Team_Turnover_Rate,Vendor_Reliability_Score,...,Change_Control_Maturity_Formal,Risk_Management_Maturity_Advanced,Risk_Management_Maturity_Basic,Risk_Management_Maturity_Formal,Team_Colocation_Fully Remote,Team_Colocation_Hybrid,Team_Colocation_Partially Colocated,Documentation_Quality_Excellent,Documentation_Quality_Good,Documentation_Quality_Poor
Team_Size,1.000000,0.885798,0.864391,0.478861,0.785608,-0.006827,0.334217,0.018489,0.014237,-0.018919,...,-0.040796,-0.021292,0.018464,-0.040968,0.041999,0.001167,-0.016725,0.072213,0.007925,-0.036334
Project_Budget_USD,0.885798,1.000000,0.864842,0.587870,0.886197,-0.002166,0.409297,0.012245,0.023810,-0.022124,...,-0.034078,-0.018464,0.020928,-0.041487,0.015319,0.000606,0.005824,0.049224,0.009986,-0.029253
Estimated_Timeline_Months,0.864391,0.864842,1.000000,0.792578,0.770566,-0.001003,0.553153,0.009925,0.034673,-0.017852,...,-0.014303,-0.019626,0.009946,-0.029473,0.027830,0.003870,-0.011733,0.037243,-0.001281,-0.018020
Complexity_Score,0.478861,0.587870,0.792578,1.000000,0.524190,0.009751,0.704350,-0.005723,0.028492,-0.012143,...,0.018353,0.003401,-0.002890,-0.021159,-0.022373,0.015837,0.010789,-0.004270,-0.002912,-0.014255
Stakeholder_Count,0.785608,0.886197,0.770566,0.524190,1.000000,-0.009377,0.368666,0.017496,0.008317,-0.020938,...,-0.030313,-0.015306,0.029050,-0.044743,0.005285,0.007219,0.004595,0.056248,0.009601,-0.040513
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
Team_Colocation_Hybrid,0.001167,0.000606,0.003870,0.015837,0.007219,-0.012359,0.013201,0.032891,0.020672,-0.003778,...,0.019440,0.006898,-0.009507,0.019022,-0.381204,1.000000,-0.373434,-0.025085,0.029138,-0.031985
Team_Colocation_Partially Colocated,-0.016725,0.005824,-0.011733,0.010789,0.004595,0.015147,0.017831,-0.029149,0.001415,0.005237,...,0.004214,-0.026033,0.005941,-0.000824,-0.335747,-0.373434,1.000000,0.005682,0.012112,0.008081
Documentation_Quality_Excellent,0.072213,0.049224,0.037243,-0.004270,0.056248,0.024500,0.005064,0.020775,-0.010788,-0.005621,...,0.018123,0.015801,-0.008454,0.000792,0.023362,-0.025085,0.005682,1.000000,-0.308429,-0.183782
Documentation_Quality_Good,0.007925,0.009986,-0.001281,-0.002912,0.009601,-0.006698,0.022114,-0.016934,-0.025303,0.005351,...,-0.004030,0.008566,-0.024109,0.019153,-0.039505,0.029138,0.012112,-0.308429,1.000000,-0.309017


In [7]:
#split into training set and test
from sklearn.model_selection import train_test_split
X_train, X_test, Y_train, Y_test = train_test_split(X_encoded, Y_encoded, test_size = 1/3, random_state =42)
X_train, X_test, Y_train, Y_test

(      Team_Size  Project_Budget_USD  Estimated_Timeline_Months  \
 1251       35.0          2360835.01                       25.0   
 365        18.0          1814824.23                       23.0   
 1366       22.0           772811.99                       21.0   
 276        18.0          1495394.30                       23.0   
 2639        6.0           613149.17                        8.0   
 ...         ...                 ...                        ...   
 1130        8.0           411443.47                        8.0   
 1294       10.0           545103.53                       14.0   
 860        14.0          1115158.11                       17.0   
 3507       28.0          1772601.08                       17.0   
 3174        9.0           588795.51                       11.0   
 
       Complexity_Score  Stakeholder_Count  Past_Similar_Projects  \
 1251              5.84               17.0                    5.0   
 365               8.56               17.0              

In [8]:
from sklearn.preprocessing import StandardScaler
sc = StandardScaler()
X_train = sc.fit_transform(X_train)
X_test = sc.transform(X_test)

In [9]:
from lightgbm  import LGBMClassifier
from sklearn.model_selection import GridSearchCV

param_grid = {'criterion':['gini','entropy'],
              'max_features': ['auto','sqrt','log2'],
              'n_estimators':[10,100]} 
grid = GridSearchCV(LGBMClassifier(), param_grid, refit = True, verbose = 3,n_jobs=-1,scoring='f1_weighted') 
   
# fitting the model for grid search 
grid.fit(X_train, Y_train) 

Fitting 5 folds for each of 12 candidates, totalling 60 fits


C:\Anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py:136: UserWarning: Could not find the number of physical cores for the following reason:
[WinError 2] The system cannot find the file specified
Returning the number of logical cores instead. You can silence this warning by setting LOKY_MAX_CPU_COUNT to the number of cores you want to use.
  warnings.warn(
  File "C:\Anaconda3\Lib\site-packages\joblib\externals\loky\backend\context.py", line 257, in _count_physical_cores
    cpu_info = subprocess.run(
               ^^^^^^^^^^^^^^^
  File "C:\Anaconda3\Lib\subprocess.py", line 548, in run
    with Popen(*popenargs, **kwargs) as process:
         ^^^^^^^^^^^^^^^^^^^^^^^^^^^
  File "C:\Anaconda3\Lib\subprocess.py", line 1026, in __init__
    self._execute_child(args, executable, preexec_fn, close_fds,
  File "C:\Anaconda3\Lib\subprocess.py", line 1538, in _execute_child
    hp, ht, pid, tid = _winapi.CreateProcess(executable, args,
                       ^^^^^^^^^^^^^^

[LightGBM] [Warning] Unknown parameter: criterion
[LightGBM] [Warning] Unknown parameter: max_features
[LightGBM] [Warning] Unknown parameter: criterion
[LightGBM] [Warning] Unknown parameter: max_features
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.009173 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 2461
[LightGBM] [Info] Number of data points in the train set: 2666, number of used features: 98
[LightGBM] [Info] Start training from score -1.710390
[LightGBM] [Info] Start training from score -1.309083
[LightGBM] [Info] Start training from score -1.602336
[LightGBM] [Info] Start training from score -1.056381


GridSearchCV(estimator=LGBMClassifier(), n_jobs=-1,
             param_grid={'criterion': ['gini', 'entropy'],
                         'max_features': ['auto', 'sqrt', 'log2'],
                         'n_estimators': [10, 100]},
             scoring='f1_weighted', verbose=3)

In [10]:
# print best parameter after tuning 
print(grid.best_params_) 
result=grid.cv_results_
print(result)
grid_predictions = grid.predict(X_test) 
   

from sklearn.metrics import confusion_matrix
cm = confusion_matrix(Y_test, grid_predictions)



# print classification report 
from sklearn.metrics import classification_report
clf_report = classification_report(Y_test, grid_predictions)

{'criterion': 'gini', 'max_features': 'auto', 'n_estimators': 100}
{'mean_fit_time': array([ 6.69073725, 39.41296659,  6.56731935, 37.71529431,  5.63962283,
       38.59435124,  5.65765219, 35.46912956,  5.81380119, 34.52949772,
        6.41934061, 23.82815051]), 'std_fit_time': array([0.04849676, 2.62425885, 0.7089684 , 2.65635447, 0.36494328,
       2.84672553, 0.29239571, 0.13909243, 0.16571655, 2.33531888,
       0.60820707, 0.75858466]), 'mean_score_time': array([0.03426895, 0.04532504, 0.03577003, 0.04116955, 0.02914662,
       0.03384604, 0.03166676, 0.03770089, 0.0282958 , 0.031706  ,
       0.03576159, 0.02942958]), 'std_score_time': array([0.00453575, 0.0058113 , 0.00252813, 0.00593597, 0.00466276,
       0.00445039, 0.00444217, 0.00626683, 0.00660325, 0.00334535,
       0.0073107 , 0.00506335]), 'param_criterion': masked_array(data=['gini', 'gini', 'gini', 'gini', 'gini', 'gini',
                   'entropy', 'entropy', 'entropy', 'entropy', 'entropy',
                   'en

In [11]:
from sklearn.metrics import f1_score
f1_macro=f1_score(Y_test,grid_predictions,average='weighted')
print("The f1_macro value for best parameter {}:".format(grid.best_params_),f1_macro)

The f1_macro value for best parameter {'criterion': 'gini', 'max_features': 'auto', 'n_estimators': 100}: 0.579118268159324


In [12]:
print("The confusion Matrix:\n",cm)

The confusion Matrix:
 [[151  96   1  32]
 [ 37 152   6 121]
 [  1   6 148 114]
 [  3  87  61 318]]


In [13]:
print("The report:\n",clf_report)

The report:
               precision    recall  f1-score   support

           0       0.79      0.54      0.64       280
           1       0.45      0.48      0.46       316
           2       0.69      0.55      0.61       269
           3       0.54      0.68      0.60       469

    accuracy                           0.58      1334
   macro avg       0.62      0.56      0.58      1334
weighted avg       0.60      0.58      0.58      1334



In [14]:
from sklearn.metrics import roc_auc_score

y_prob = grid.predict_proba(X_test)

roc_auc = roc_auc_score(
    Y_test,
    y_prob,
    multi_class="ovr",   # or "ovo"
    average="weighted"
)

print("ROC AUC:", roc_auc)

[LightGBM] [Warning] Unknown parameter: criterion
[LightGBM] [Warning] Unknown parameter: max_features
ROC AUC: 0.8195245332913398


In [15]:
best_model = grid.best_estimator_
importance_of_each_feature=best_model.feature_importances_
importance_of_each_feature

array([204, 304, 174, ...,  63,  41,  67])